In [3]:
import carla
import random
import time
import logging
import os
import signal
import numpy as np
import tempfile
import pickle
from typing import Callable, Dict, Set, Tuple, Any

from norospy import ROSFoxgloveClient

from mmdet3d.apis import MonoDet3DInferencer

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(message)s')

In [4]:
client = carla.Client("localhost", 2000)

In [5]:
world = client.load_world("Town03")

In [6]:
bp_lib = world.get_blueprint_library()

In [7]:
vehicle_bp = bp_lib.filter("vehicle")
spawn_pts = world.get_map().get_spawn_points()

In [8]:
ego_vehicle = world.spawn_actor(random.choice(vehicle_bp), random.choice(spawn_pts))

In [9]:
# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_bp), random.choice(spawn_pts))

In [10]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

In [11]:
spectator = world.get_spectator()

In [12]:
transform = ego_vehicle.get_transform()
spectator.set_transform(carla.Transform(transform.location + carla.Location(z=2), carla.Rotation(yaw=90)))

In [13]:
cam_bp = None
# TODO: set proper transformation to the cam
cam_init_transform = carla.Transform(carla.Location(z=3))
cam_bp = world.get_blueprint_library().find("sensor.camera.rgb")
cam = world.spawn_actor(cam_bp, cam_init_transform, attach_to=ego_vehicle)
# cam.attributes

In [14]:
# initialize the model
config_path = "/home/ws/uqmfs/mmdetection3d/configs/fcos3d/fcos3d_r101-caffe-dcn_fpn_head-gn_8xb2-1x_nus-mono3d_finetune.py"
checkpoint_path = "/home/ws/uqmfs/mmdetection3d/weights/fcos3d_r101_caffe_fpn_gn-head_dcn_2x8_1x_nus-mono3d_finetune_20210717_095645-8d806dc2.pth"

inferencer = MonoDet3DInferencer(config_path, checkpoint_path)

Loads checkpoint by local backend from path: /home/ws/uqmfs/mmdetection3d/weights/fcos3d_r101_caffe_fpn_gn-head_dcn_2x8_1x_nus-mono3d_finetune_20210717_095645-8d806dc2.pth
05/30 16:38:03 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/ws/uqmfs/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [15]:
def get_anotation(camera_matrix):
    infos = {}
    infos={
        'data_list': [
            {
                'images': {
                    'CAM2': {
                        'img_path': None,
                        'cam2img': camera_matrix,
                        'lidar2img': np.array([]),
                        'lidar2cam': np.array([]),
                    }
                }
            }
        ]
    }
    infos_file = tempfile.NamedTemporaryFile(delete=False, suffix='.pkl')
    pickle.dump(infos, infos_file)
    infos_file.close()
    return infos_file

In [23]:
def online_inference(msg: Any, ts: int):
    logging.info('Now here we are!')
    # TODO: convert ros image msg to ndarray
    img = ROSFoxgloveClient.parse_ros_image(msg)
    # projection/camera matrix from ros topic
    # P = np.array([400.00000000000006, 0, 400, 0, 0, 400.00000000000006, 300, 0, 0, 0, 1, 0], dtype=np.float64).reshape(3,4)
    C = np.array([400.00000000000006, 0, 400, 0, 400.00000000000006, 300, 0, 0, 1], dtype=np.float64).reshape(3,3)
    
    # TODO: construct the pkl file based on the camera calibration
    infos_file = get_anotation(camera_matrix=C)

    input = dict(img=img, infos=infos_file.name)
    results = inferencer(input, return_datasamples=False, return_vis=True, out_dir='/temp')

In [24]:
try:
    client = ROSFoxgloveClient('ws://localhost:8765')
    client.run_background()
    client.subscribe('/carla/autopilot/front/image', 'sensor_msgs/Image', online_inference)
    signal.pause()

except NotImplementedError:
    pass
finally:
    client.close()

INFO:root:Loaded 124 message definitions
INFO:root:Connecting ...
INFO:root:Connected.
INFO:root:Now here we are!


Output()

Exception in thread Thread-777:
Traceback (most recent call last):
  File "/home/ws/uqmfs/miniconda3/envs/openmmlab/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ws/uqmfs/miniconda3/envs/openmmlab/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ws/uqmfs/miniconda3/envs/openmmlab/lib/python3.8/site-packages/norospy/ros.py", line 121, in run
    callback(decoder(payload), ts)
  File "/tmp/ipykernel_929149/2177829156.py", line 13, in online_inference
  File "/home/ws/uqmfs/mmdetection3d/mmdet3d/apis/inferencers/base_3d_inferencer.py", line 215, in __call__
    results = self.postprocess(preds, visualization,
  File "/home/ws/uqmfs/mmdetection3d/mmdet3d/apis/inferencers/base_3d_inferencer.py", line 273, in postprocess
    result = self.pred2dict(pred, pred_out_dir)
  File "/home/ws/uqmfs/mmdetection3d/mmdet3d/apis/inferencers/base_3d_inferencer.py", line 336, in pred2dict
    img_path = osp.

KeyboardInterrupt: 

In [ ]:
cam_destroyed_sucessfully = cam.destroy()